# The part of Fermi-Hubbard Model

## 1, Change the number operator $n_i=c^{\dagger}c$ into $(n_i-\frac{1}{2})$

Firstly, if we change the number operator in such way, the eigenstates of the Hamiltonian keep the same. I have read a paper, in which the author mentioned such alternative would lead to a narrower error bound.

Therefore, to change the number operator, I just create the class "ModifiedNumOp", which contains all similar properties as "NumberOp"

In [ ]:
class ModifiedNumOp(HamiltonianOp):
    """
    Number operator :math:`n_{i\sigma}-Identity/2`.
    """
    def __init__(self,i:Sequence[int], s:int, coeff:float, mod:int =0) -> None:
        self.i = tuple(i)
        assert s in [0, 1]
        self.s = s
        self.coeff = coeff
        self.mod = mod  # mod = 0, indicated the operator belongs to Fermi_Hubbard model
    
    def __neg__(self):
        """
        Logical negation.
        """
        return ModifiedNumOp(self.i, self.s, -self.coeff, self.mod)
    
    def __rmul__(self, other):
        """
        Logical scalar product.
        """
        if not isinstance(other, (Rational, float)):
            raise ValueError("expecting a scalar argument")
        return ModifiedNumOp(self.i, self.s, other * self.coeff, self.mod)

    def __add__(self, other):
        """
        Logical sum.
        """
        if isinstance(other, ZeroOp):
            return self
        if not self.proportional(other):
            raise ValueError("can only add another number operator acting on same site")
        return ModifiedNumOp(self.i, self.s, self.coeff + other.coeff, self.mod)

    def __sub__(self, other):
        """
        Logical difference.
        """
        return self + (-other)

    def __str__(self) -> str:
        """
        Represent the operator as a string.
        """
        c = "" if self.coeff == 1 else f"({self.coeff}) "
        return c + f"Mn_{{{self.i}, {'up' if self.s == 0 else 'dn'}}}"

    def __eq__(self, other) -> bool:
        """
        Equality test.
        """
        if isinstance(other, ModifiedNumOp):
            if self.i == other.i and self.s == other.s and self.coeff == other.coeff:
                return True
        return False

    def __lt__(self, other) -> bool:
        """
        "Less than" comparison, used for, e.g., sorting a sum of operators.
        """
        assert isinstance(other, HamiltonianOp)
        if isinstance(other, (ProductOp, SumOp)):
            return True
        if isinstance(other, (ZeroOp, PauliOp, HoppingOp, AntisymmHoppingOp, NumberOp)):
            return False # in fact, ModifiedNumOp will not meet NumberOp in one model
        assert isinstance(other, ModifiedNumOp)
        # lexicographical comparison
        return (self.s, self.i, self.coeff) < (other.s, other.i, other.coeff)

    def proportional(self, other) -> bool:
        """
        Whether current operator is equal to 'other' up to a scalar factor.
        """
        if isinstance(other, NumberOp):
            if self.i == other.i and self.s == other.s:
                return True
        return False

    def as_field_operator(self) -> FieldOp:
        """
        To transfer the HamiltonianOp into FieldOp.
        Here we specify the mod as 0.
        """
        if self.mod ==0:
            return FieldOp([
            ProductFieldOp([
                ElementaryFieldOp(FieldOpType.FERMI_MODNUM, self.i, self.s)], self.coeff)])
        return FieldOp_FB([
            ProductFieldOp_FB([
                ElementaryFieldOp_FB(FieldOpType_FB.FERMI_MODNUM, self.i, self.s)], self.coeff)])

    def support(self) -> list[tuple]:
        """
        This function is to return the index and spin of the operator
        """
        return [self.i + (self.s,)]

    def translate(self, shift: Sequence[int]):
        """
        Translate by `shift` and return the translated operator.
        """
        return ModifiedNumOp(tuple(x + s for x, s in zip(self.i, shift)), self.s, self.coeff, self.mod)

    def normalize(self) -> tuple:
        """
        Return a normalized copy of the operator together with its scalar prefactor.
        """
        if self.coeff == 1:
            return self, 1
        return ModifiedNumOp(self.i, self.s, 1, self.mod), self.coeff

    def is_zero(self) -> bool:
        """
        Indicate whether the operator acts as zero operator.
        """
        return self.coeff == 0

    @property
    def fermi_weight(self) -> int:
        """
        Maximum number of fermionic creation and annihilation operators multiplied together.
        """
        return 2

    def norm_bound(self) -> float:
        """
        Upper bound on the spectral norm of the operator.
        n-1/2 = -1/2*pauliZ
        """
        return abs(self.coeff*0.5)
    
    def set_mod(self, mod:int = 1):
        """
        A mutator function
        """
        if self.s==2:
            self.mod = 1
            return
        assert mod in [0,1]
        self.mod = mod

    def Mod2Num(self) -> NumberOp:
        """
        Being used to compute the commutator between (n-1/2) and other operators
        """
        return NumberOp(self.i, self.s, self.coeff, self.mod)

## 2, A proparate FieldOp for ModifiedNumOp
similar to the original "NumberOp", we need to generate the list of "ModifiedNumOp" for each site of the model. Therefore, when constructing such list for "NumberOp", we could just use the element to deduct 1/2*Id. Or we could just insert the -PauliZ/2 during the JW transformation.

In [ ]:
def  construct_fermionic_operators(nmodes: int):
    """
    Generate sparse matrix representations of the fermionic creation and
    annihilation operators for `nmodes` modes (or sites),
    based on Jordan-Wigner transformation.
    """
    I = sparse.identity(2)
    Z = sparse.csr_matrix([[ 1.,  0.], [ 0., -1.]])
    U = sparse.csr_matrix([[ 0.,  0.], [ 1.,  0.]])
    clist = []
    mlist = [] # indeed 1/2 * -Z
    bclist = []
    for i in range(nmodes):
        c = sparse.identity(1)
        m = sparse.identity(1)
        bc = sparse.identity(1)
        for j in range(nmodes):
            if j < i:
                c = sparse.kron(c, I)
                m = sparse.kron(m, I)
                bc = sparse.kron(bc, I)
            elif j == i:
                c = sparse.kron(c, U)
                m = sparse.kron(m, Z)*(-.5) # n-1/2 = -Z/2
                bc = sparse.kron(bc, U)
            else:
                c = sparse.kron(c, Z)
                m = sparse.kron(m, I)
                bc = sparse.kron(bc, I)
        c = sparse.csr_matrix(c)
        m = sparse.csr_matrix(m)
        bc = sparse.csr_matrix(bc)
        c.eliminate_zeros()
        m.eliminate_zeros()
        bc.eliminate_zeros()
        clist.append(c)
        mlist.append(m)
        bclist.append(bc)
    # corresponding annihilation operators
    alist = [sparse.csr_matrix(c.conj().T) for c in clist]
    balist = [sparse.csr_matrix(bc.conj().T) for bc in bclist]
    # corresponding number operators
    nlist = []
    for i in range(nmodes):
        f = 1 << (nmodes - i - 1)
        data = [1. if (n & f == f) else 0. for n in range(2**nmodes)]
        nlist.append(sparse.dia_matrix((data, 0), 2*(2**nmodes,)))
    return clist, alist, nlist, mlist, bclist, balist

## 3, the commutator of ModifiedNumOp and other operators
Because $$[(n-1/2), O]=(n-1/2)O-O(n-1/2)=nO-On-O/2+O/2=nO-On=[n,O]$$ we use the Mod2Num() function to transfer the ModifiedNumOp the corresponding NumberOp with the same properties. Then apply the NumberOp to compute the commutator. I provided a segement code as an example.

In [ ]:
if isinstance(a, HoppingOp):
        if isinstance(b, HoppingOp):
            return _commutator_hopping(a, b)
        if isinstance(b, AntisymmHoppingOp):
            return _commutator_mixed_symm_hopping(a, b)
        if isinstance(b, NumberOp):
            return _commutator_hopping_number(a, b)
        if isinstance(b, ModifiedNumOp):
            return _commutator_hopping_number(a, b.Mod2Num())
        if isinstance(b, PauliOp):
            return ZeroOp()
    elif isinstance(a, AntisymmHoppingOp):
        if isinstance(b, HoppingOp):
            # pylint: disable=arguments-out-of-order
            return -_commutator_mixed_symm_hopping(b, a)
        if isinstance(b, AntisymmHoppingOp):
            return _commutator_antisymm_hopping(a, b)
        if isinstance(b, NumberOp):
            return _commutator_antisymm_hopping_number(a, b)
        if isinstance(b, ModifiedNumOp):
            return _commutator_antisymm_hopping_number(a, b.Mod2Num())
        if isinstance(b, PauliOp):
            return ZeroOp()
    """......"""

# The part of Hubbard-Holstein Model
Recall the Hamiltonian of the Hubbard-Holstein Model: 
$$H_{Holstein} = H_{Fermi_Hubbard}+n_{boson}+n_{fermi}*(b_{i}^{\dagger}+b_{i})$$

## 1, Create operators for bosons
When computing the net commutator between $(b_{i}^{\dagger}+b_i=\mathbb{X})$ and $n_{boson}=-\mathbb{Z}/2+1/2$, all results are Pauli Matrix. Then we just create a set of PauliOp for bosons.

In [ ]:
class PauliOp(HamiltonianOp):
    """
    Identity Operator. i.e. c*I, c is coefficiency
    """
    def __init__(self,i: Sequence[int],cate: int, coeff: float) -> None:
        super().__init__()
        """
        Category: 0-Identity, 1-pauliX, 2-i*pauliY, 3-pauliZ
        """
        assert cate in [0,1,2,3]
        self.i = i
        self.cate = cate
        self.coeff = coeff
    
    def __neg__(self):
        """
        Logical negation.
        """
        return PauliOp(self.i, self.cate, -self.coeff)

    def __rmul__(self, other):
        """
        Logical scalar product.
        """
        if not isinstance(other, (Rational, float)):
            raise ValueError("expecting a scalar argument")
        return PauliOp(self.i, self.cate, other*self.coeff)
    
    def __add__(self, other):
        """
        Logical sum.
        """
        if isinstance(other, ZeroOp):
            return self
        if not self.proportional(other):
            raise ValueError("can only add another product operator with same factors")
        return PauliOp(self.i, self.cate,self.coeff+other.coeff)
    
    def __sub__(self, other):
        """
        Logical difference.
        """
        return self+ (-other)
    
    def __str__(self) -> str:
        c = "" if self.coeff ==1 else f"{self.coeff}"
        match self.cate:
            case 1:
                lb = "X"
            case 2:
                lb = 'iY'
            case 3:
                lb = "Z"
            case _:
                lb = "Id"
        return c+f"{lb}"
    
    def __eq__(self, other) -> bool:
        """
        Equality test.
        """
        return isinstance(other, PauliOp) and self.coeff==other.coeff
    
    def __lt__(self, other) -> bool:
        """
        "Less than" comparison, used for, e.g., sorting a sum of operators.
        """
        assert isinstance(other, HamiltonianOp)
        if isinstance(other,ZeroOp):
            return True
        if isinstance(other, (HoppingOp, AntisymmHoppingOp, NumberOp, ModifiedNumOp, ProductOp, SumOp)):
            return False
        assert isinstance(other, PauliOp)
        return self.coeff<other.coeff
    
    def proportional(self, other) -> bool:
        """
        Whether current operator is equal to 'other' up to a scalar factor.
        """
        if isinstance(other, PauliOp):
            return self.cate == other.cate
        return False
    
    def as_field_operator(self) -> FieldOp_FB:
        """
        The "_FB" indicates the model is Hubbard-Holstein model, which contains both Fermion
        and Boson.
        """
        match self.cate:
            case 1: # pauli X = b^{\dagger} + b
                op_list = [ProductFieldOp_FB([ElementaryFieldOp_FB(FieldOpType_FB.BOSON_CREATE, self.i, 2)], self.coeff), 
                ProductFieldOp_FB([ElementaryFieldOp_FB(FieldOpType_FB.BOSON_ANNIHIL, self.i, 2)], self.coeff)]
            case 2: # i*pauli Y = -b^{\dagger} + b
                op_list = [ProductFieldOp_FB([ElementaryFieldOp_FB(FieldOpType_FB.BOSON_CREATE, self.i, 2)], -self.coeff), 
                ProductFieldOp_FB([ElementaryFieldOp_FB(FieldOpType_FB.BOSON_ANNIHIL, self.i, 2)], self.coeff)]
            case 3: # pauli Z = -2(n-1/2)
                op_list = [ProductFieldOp_FB([ElementaryFieldOp_FB(FieldOpType_FB.FERMI_MODNUM, self.i, 2)], -2.*self.coeff)]
            case _: # Identity
                raise RuntimeError("Identity operator should be emitted")
        return FieldOp_FB(op_list)
    
    def support(self) -> list[tuple]:
        """
        It should be just return the position and the spin acting by the operator
        """
        return [self.i + (2,)]

    def translate(self, shift: Sequence[int]):
        """
        move the position into correct representation of the lattice
        """
        return PauliOp(tuple(x + s for x, s in zip(self.i, shift)), self.cate, self.coeff)
    
    def normalize(self) -> tuple:
        return PauliOp(self.i, self.cate,1.), self.coeff
    
    def is_zero(self) -> bool:
        return self.coeff==0
    
    @property
    def fermi_weight(self) -> int:
        return 2
    
    @property
    def mod(self) -> int:
        return 1

    def norm_bound(self) -> float:
        return abs(self.coeff)
    
    def set_mod(self, mod:int):
        """
        For pauli matrix, it's applied only in Hubbard-Holstein model, so we do not change
        it's mod.
        """
        return

## 2, Some changes for FieldOp
As we introduced Bosons as particles with a new type of spins into the model, we need to rearange the way to construct FieldOp. For each elementary operators in Holstein model, we could apply the class below:

In [ ]:
class ElementaryFieldOp_FB(ElementaryFieldOp):
    def __init__(self, otype: FieldOpType_FB, i: Sequence[int], s: int):
        self.otype = otype
        self.i = tuple(i)
        assert s in [0, 1, 2]
        self.s = s
    
    def __str__(self) -> str:
        nt = 'up'
        if self.s==1:
            nt = 'dn'
        elif self.s==2:
            nt = 'bn'
        if self.otype == FieldOpType_FB.FERMI_CREATE:
            return f"ad_{{{self.i}, {nt}}}"
        if self.otype == FieldOpType_FB.FERMI_ANNIHIL:
            return f"a_{{{self.i}, {nt}}}"
        if self.otype == FieldOpType_FB.FERMI_NUMBER:
            return f"n_{{{self.i}, {nt}}}"
        if self.otype == FieldOpType_FB.FERMI_MODNUM:
            return f"mn_{{{self.i}, {nt}}}"
        if self.otype == FieldOpType_FB.BOSON_CREATE:
            return f"bc_{{{self.i}, {nt}}}"
        if self.otype == FieldOpType_FB.BOSON_ANNIHIL:
            return f"ba_{{{self.i}, {nt}}}"
        assert False

## 3, To construct the matrix form of the operator in Holstein Model
For Fermi-Hubbard Model, the structure of the matrix form is denoted as:
$$H=\sum H_{prod}$$
where $$H_{prod}=(h_{0,up}\otimes h_{0,dn})\otimes \dots$$
Similarly, we could construct the matrix for Holstein Model in the following form:
$$H_{prod}=(h_{0,up}\otimes h_{0,dn}\otimes h_{0,boson})\otimes \dots$$

In [ ]:
def as_matrix(self, latt_shape: Sequence[int], translatt: SubLattice = None):
        # num of lattice sites
        # we consider the bosons as a 'Special' spin-status
        # Therefore, there factor 3 from spin (up, down, boson)
        L = 3* math.prod(latt_shape)
        if not self.terms:
            # fast-return zero matrix if terms are empty
            return sparse.csr_matrix((2**L, 2**L))
        clist, alist, nlist, mlist, bclist, balist = construct_fermionic_operators(L)
        mat = 0
        if translatt is None:
            tps = [len(latt_shape) * (0,)]
        else:
            tps = translatt.instantiate(len(latt_shape) * (0,), latt_shape)
        for tp in tps:
            for term in self.terms:
                fstring = sparse.identity(2**L)
                for op in term.ops:
                    # take spin into account for indexing
                    j = 3 * latt_coord_to_index(tuple(x + y for x, y in zip(op.i, tp)), latt_shape) + op.s
                    if op.otype == FieldOpType_FB.FERMI_CREATE:
                        fstring = fstring @ clist[j]
                    elif op.otype == FieldOpType_FB.FERMI_ANNIHIL:
                        fstring = fstring @ alist[j]
                    elif op.otype == FieldOpType_FB.FERMI_NUMBER:
                        fstring = fstring @ nlist[j]
                    elif op.otype == FieldOpType_FB.FERMI_MODNUM:
                        fstring = fstring @ mlist[j]
                    elif op.otype == FieldOpType_FB.BOSON_CREATE:
                        fstring = fstring @ bclist[j]
                    elif op.otype == FieldOpType_FB.BOSON_ANNIHIL:
                        fstring == fstring @ balist[j]
                    else:
                        raise RuntimeError(f"unexpected fermionic operator type {op.otype}")
                mat += float(term.coeff) * fstring
        return mat

## 4, unsolved problem
To read the notebook "Hubbard_Holstein_Model.ipynb"